# 基因相关性分析完整教程

本教程展示了如何使用 `gene_correlation_ultra` 进行空间转录组数据的基因相关性分析。

## 目录
1. [数据准备](#1-数据准备)
2. [全部样本分析](#2-全部样本分析)
3. [分组分析](#3-分组分析)
4. [单个样本分析](#4-单个样本分析)
5. [可视化](#5-可视化)

---

## 1. 数据准备

首先导入必要的库并加载数据。

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import sys
import os
import gc

from gene_correlation_ultra import gene_correlation_ultra, GeneCorrelationUltra

In [ ]:
# 加载 AnnData 数据
adata = sc.read_h5ad("../subcluster/Malignan.NMF.allsample.h5ad")

# 查看分组信息
print("分组信息:", adata.obs["group"].unique())

In [ ]:
# 加载目标基因列表
target_gene = pd.read_csv("./geneList", sep="\t", index_col=None)
print(f"目标基因数量: {len(target_gene)}")

# 加载差异表达基因列表
deg = pd.read_csv("../../../05.group2DEGs/DEG/de_wilcoxon.group.Responder-vs-Non-responder.sig_filtered.csv")
print(f"差异基因数量: {len(deg)}")

In [ ]:
# 提取基因数组
target_genes = target_gene["Gene"].to_numpy()
de_genes = deg["geneName"].to_numpy()

print(f"目标基因: {len(target_genes)} 个")
print(f"差异基因: {len(de_genes)} 个")

---

## 2. 全部样本分析

使用所有样本进行相关性分析。

In [ ]:
# 释放内存
gc.collect()

# 执行分析
corr_df, pval_df, sig_pairs = gene_correlation_ultra(
    st_expr_matrix=adata,
    target_genes=target_genes,
    de_genes=de_genes,
    method="spearman",          # 相关性方法: spearman, pearson, kendall
    p_adjust="fdr_bh",           # P值校正方法
    threshold_p=0.05,             # P值显著性阈值
    batch_size=5000,              # 批处理大小
    n_workers=16,                 # 并行进程数
    enable_numba=True,            # 启用 Numba 加速
    min_corr_threshold=0,         # 最小相关性阈值
    max_memory_mb=50720,          # 最大内存限制 (MB)
    verbose=True,                 # 显示详细日志
    output_dir='results/all/spearman/',
    save_full_matrices=True,      # 保存完整矩阵
    matrix_format="csv.gz"        # 矩阵保存格式
)

In [ ]:
# 查看结果统计
print(f"显著相关对数量: {len(sig_pairs)}")
print(f"\n前10个显著相关对:")
print(sig_pairs.head(10))

---

## 3. 分组分析

按照分组（如响应者 vs 非响应者）分别进行相关性分析。

In [ ]:
# 遍历每个分组
for group in adata.obs["group"].unique():
    print(f"\n=== 分析分组: {group} ===")
    
    # 提取当前分组的数据
    adata_group = adata[adata.obs["group"] == group].copy()
    
    # 执行分析
    corr_df, pval_df, sig_pairs = gene_correlation_ultra(
        st_expr_matrix=adata_group,
        target_genes=target_genes,
        de_genes=de_genes,
        method="spearman",
        p_adjust="fdr_bh",
        threshold_p=0.05,
        batch_size=5000,
        n_workers=16,
        enable_numba=True,
        min_corr_threshold=0,
        max_memory_mb=50720,
        verbose=True,
        output_dir=f'results/group/spearman/{group}',
        save_full_matrices=True,
        matrix_format="csv.gz"
    )
    
    print(f"分组 {group}: 找到 {len(sig_pairs)} 个显著相关对")

---

## 4. 单个样本分析

按照单个样本分别进行相关性分析。

In [ ]:
# 遍历每个样本
for sample in adata.obs["sample_id"].unique():
    print(f"\n=== 分析样本: {sample} ===")
    
    # 提取当前样本的数据
    adata_sample = adata[adata.obs["sample_id"] == sample].copy()
    
    # 执行分析
    corr_df, pval_df, sig_pairs = gene_correlation_ultra(
        st_expr_matrix=adata_sample,
        target_genes=target_genes,
        de_genes=de_genes,
        method="spearman",
        p_adjust="fdr_bh",
        threshold_p=0.05,
        batch_size=5000,
        n_workers=16,
        enable_numba=True,
        min_corr_threshold=0,
        max_memory_mb=50720,
        verbose=True,
        output_dir=f'results/sample/spearman/{sample}',
        save_full_matrices=True,
        matrix_format="csv.gz"
    )
    
    print(f"样本 {sample}: 找到 {len(sig_pairs)} 个显著相关对")

---

## 5. 可视化

使用 CorrelationVisualizer 进行结果可视化。

In [ ]:
from gene_correlation_ultra.gene_correlation_plot import CorrelationVisualizer

In [ ]:
# 创建可视化器
visualizer = CorrelationVisualizer(
    st_expr_matrix=adata,
    output_dir="correlation_plots"
)

### 5.1 单基因对散点图

In [ ]:
# 绘制单个基因对的散点图
visualizer.plot_single_pair_scatter(
    gene1="XPO1",
    gene2="DLEU1",
    show_plot=True,
    method="spearman"
)

### 5.2 其他可视化选项

更多可视化功能请参考 `docs/PLOT_USAGE.md` 文档：

- **Top N 基因对网格图**: `visualizer.plot_top_pairs_scatter_grid()`
- **多基因对热图**: `visualizer.plot_multiple_pairs_heatmap()`
- **完整相关性矩阵热图**: `visualizer.plot_correlation_matrix_heatmap()`

---

## 总结

本教程展示了基因相关性分析的完整流程：

1. ✅ 数据准备：加载 AnnData、目标基因和差异基因
2. ✅ 全部样本分析：对所有样本进行统一分析
3. ✅ 分组分析：按分组分别分析
4. ✅ 单个样本分析：按样本分别分析
5. ✅ 可视化：生成散点图、热图等

更多参数配置和高级用法请参考项目 README.md 和 API 文档。